# 非线性时程分析使用指南
本笔记展示如何使用 `opss_frame` 中的 API 完成增量动力分析（IDA）和多条地震动条带分析（MSA）。

In [ ]:
from pathlib import Path
import pickle
import numpy as np

repo_root = Path('..').resolve() if (Path.cwd() / 'examples').exists() else Path.cwd()
examples_dir = repo_root / 'examples'
inputs_dir = examples_dir / 'Inputs'
records_dir = examples_dir / 'recordsMSA'


## 准备 IDA 所需的模型与输入
`RCMRF` 类负责建立框架模型并协调分析流程。配置时需要提供截面、荷载、材料以及地震动信息，并明确 `periods_ida` 与阻尼比。

In [ ]:
import pickle
from rcmrf import RCMRF

sections_file = inputs_dir / 'hinge_models.pickle'
with sections_file.open('rb') as f:
    sections_data = pickle.load(f)

materials_file = inputs_dir / 'materials.csv'
loads_file = inputs_dir / 'action.csv'
gm_dir = examples_dir / 'gm_ida'
gm_files = ['GMR_names1.txt', 'GMR_names2.txt', 'GMR_dts.txt']

ida_periods = [0.96, 1.03]
damping_ratio = 0.05
analysis_output = examples_dir / 'outputs/ida_demo'
analysis_output.mkdir(parents=True, exist_ok=True)

ida_runner = RCMRF(sections_data, loads_file, materials_file, analysis_output,
                     gmdir=gm_dir, gmfileNames=gm_files, analysis_type=['IDA'],
                     periods_ida=ida_periods, damping=damping_ratio, flag3d=True,
                     export_at_each_step=False, analysis_time_step=0.01, max_runs=5)


## 运行 IDA 并查看输出
调用 `run_model()` 会自动建立模型、执行 IDA 并在输出目录保存强度量与逐步结果。

In [ ]:
ida_runner.wipe()
ida_runner.run_model()
ida_runner.wipe()

im_file = (analysis_output / 'IM.csv').read_text() if (analysis_output / 'IM.csv').exists() else 'IM 文件将在分析完成后生成。'
print(im_file[:200])


## 设置并执行 MSA
`MultiStripeAnalysis` 使用与 IDA 相同的模型输入，但需要将记录按照条带批次组织。我们复用 `periods_ida` 来计算圆频率。

In [ ]:
from analysis.multiStripeAnalysis import MultiStripeAnalysis, get_records

omegas = 2 * np.pi / np.array(ida_periods)
msa_output = examples_dir / 'outputs/msa_demo'
msa_output.mkdir(parents=True, exist_ok=True)

msa_runner = MultiStripeAnalysis(sections_data, loads_file, materials_file, records_dir,
                                   damping_ratio, omegas, msa_output,
                                   system='space', hingeModel='hysteretic', flag3d=True,
                                   analysis_time_step=0.01, drift_capacity=10.,
                                   export_at_each_step=True)

record_batches = get_records(records_dir, msa_output, ['GMR_H1_names.txt', 'GMR_H2_names.txt', 'GMR_dts.txt'])
# msa_runner.start_process(record_batches, processes=2)  # 视计算资源情况启用
